![alt text](https://raw.githubusercontent.com/DarleySoares/codenation_data_science/master/projeto_pratico/images/header.png)

In [53]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from sklearn.preprocessing import LabelEncoder, StandardScaler

palette = ['#F63366', '#FCE92D', '#B0A320', '#2D9AFC', '#206CB0']
sns.set_palette(palette)


# Coleta de dados

In [91]:
# leitura dos dados dos mercados
df_market = pd.read_csv('../../estaticos_market.csv')

# Análise Exploratória

In [137]:
# verificando quantidade de dados vazios por coluna
columns =list(df_market.columns)
null_data = list(df_market.isnull().sum())
df_amount_null_data = pd.DataFrame(data = [columns, null_data])
df_amount_null_data = df_amount_null_data.transpose()
df_amount_null_data.columns = ['Column', 'Amount of null data']

In [138]:
# colunas que estão completas
complete_columns = list(df_amount_null_data.Column.loc[df_amount_null_data['Amount of null data'] == 0])

# colunas que estão totamente incompletas
null_columns = list(df_amount_null_data.Column.loc[df_amount_null_data['Amount of null data'] == df_market.shape[0]])

# colunas que possuem mais de 50% dos dados incompletos
half_null_columns = list(df_amount_null_data.Column.loc[df_amount_null_data['Amount of null data'] >= 0.5 * df_market.shape[0]])

# colunas que possuem menos de 50% dos dados incompletos
less_of_half_null_columns = list(df_amount_null_data.Column.loc[df_amount_null_data['Amount of null data'] < 0.5 * df_market.shape[0]])

In [139]:
print('Colunas com dados totalmente completos: {}'.format(len(complete_columns)))
print('Colunas com dados totalmente incompletos: {}'.format(len(null_columns)))
print('Colunas com mais da metade dos dados incompletos: {}'.format(len(half_null_columns)))
print('Colunas com menos da metade dos dados incompletos: {}'.format(len(less_of_half_null_columns)))

Colunas com dados totalmente completos: 19
Colunas com dados totalmente incompletos: 5
Colunas com mais da metade dos dados incompletos: 131
Colunas com menos da metade dos dados incompletos: 51


# Pré processamento de dados

In [140]:
df = df_market.filter(items = less_of_half_null_columns)

In [145]:
# cria um dicionário com o setor mais frequente de cada ramo
agrup = dict(df.groupby(['de_ramo'])['setor'].agg(pd.Series.mode))
# completa os dados nulos com o termo mais frequente do ramo
df.setor.loc[pd.isnull(df.setor)] = df.de_ramo.map(agrup)

{'ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL': 'SERVIÇO',
 'ATIVIDADES IMOBILIARIAS': 'SERVIÇO',
 'BENS DE CONSUMO': 'INDUSTRIA',
 'COMERCIO E REPARACAO DE VEICULOS': 'COMERCIO',
 'COMERCIO POR ATACADO': 'COMERCIO',
 'COMERCIO VAREJISTA': 'COMERCIO',
 'CULTURA, ESPORTE E RECREACAO': 'SERVIÇO',
 'ELETRICIDADE E GAS': 'INDUSTRIA',
 'ELETROELETRONICOS': 'INDUSTRIA',
 'FARMACEUTICA': 'INDUSTRIA',
 'INDUSTRIA AUTOMOTIVA': 'INDUSTRIA',
 'INDUSTRIA DA CONSTRUCAO': 'CONSTRUÇÃO CIVIL',
 'INDUSTRIA DIGITAL': 'SERVIÇO',
 'INDUSTRIAS EXTRATIVAS': 'INDUSTRIA',
 'ORGANISMOS INTERNACIONAIS': 'SERVIÇO',
 'OUTROS': array([], dtype=object),
 'PAPEL E CELULOSE': 'INDUSTRIA',
 'PRODUTOS DE AGROPECUARIA': 'AGROPECUARIA',
 'QUIMICA-PETROQUIMICA': 'INDUSTRIA',
 'SEGUROS, PLANOS DE PREVIDENCIA E PLANOS DE SAUDE': 'SERVIÇO',
 'SERVICOS ADMINISTRATIVOS': 'SERVIÇO',
 'SERVICOS DE ALOJAMENTO/ALIMENTACAO': 'SERVIÇO',
 'SERVICOS DE EDUCACAO': 'SERVIÇO',
 'SERVICOS DE SANEAMENTO BASICO': 'SERVIÇO',
 'SERVICOS

In [97]:
# cria um dicionário com a divisão mais frequente de cada ramo
agrup = df.groupby(['de_ramo'])['nm_divisao'].agg(pd.Series.mode)
# completa os dados nulos com o termo mais frequente do ramo
df.nm_divisao.loc[pd.isnull(df.nm_divisao)] = df.de_ramo.map(agrup)

In [98]:
# remove a data na qual uma situação foi registrado na Receita Federal
df.drop(['dt_situacao'], axis = 1, inplace = True)

In [99]:
# cria um dicionário com o segmento mais frequente de cada ramo
agrup = df.groupby(['de_ramo'])['nm_segmento'].agg(pd.Series.mode)
# completa os dados nulos com o termo mais frequente do ramo
df.nm_segmento.loc[pd.isnull(df.nm_segmento)] = df.de_ramo.map(agrup)

In [100]:
# completa os dados nulos com o valor False, que representa não possuir estação de abastecimento próprio
df.fl_spa.fillna(False, inplace = True)

In [101]:
# completa os dados nulos com o valor False, que representa não possui certificação ANTT para transportação
df.fl_antt.fillna(False, inplace = True)

In [102]:
# completa os dados nulos com o valor zero, que representa que a empresa não possui veículos que pertencem ao grupo pesados ou leves
df.vl_total_veiculos_pesados_grupo.fillna(0, inplace = True)
df.vl_total_veiculos_leves_grupo.fillna(0, inplace = True)

# completa os dados nulos com o valor da soma do total de veículos dos grupos leves e pesados
aux = df.vl_total_veiculos_pesados_grupo + df.vl_total_veiculos_leves_grupo
df.fl_veiculo = list(map(lambda x: True if x > 0 else False, aux))

In [103]:
# exclui as colunas que não são interessantes para a análise
df.drop(columns = ['fl_optante_simples','fl_optante_simei','sg_uf_matriz','de_saude_tributaria','de_saude_rescencia','nu_meses_rescencia','idade_media_socios','idade_maxima_socios','idade_minima_socios','de_faixa_faturamento_estimado_grupo','vl_faturamento_estimado_grupo_aux','fl_passivel_iss','qt_socios_st_regular','fl_simples_irregular','empsetorcensitariofaixarendapopulacao'], axis = 1, inplace = True)

In [104]:
# completa os dados nulos com o valor zero, que representa que a empresa não possui sócios do tipo pessoa jurídico ou pessoa física.
df.qt_socios_pj.fillna(0, inplace = True)
df.qt_socios_pf.fillna(0, inplace = True)
df.qt_socios.fillna(df.qt_socios_pj+df.qt_socios_pf, inplace = True)

In [105]:
# cria um dicionário com a meso região mais frequente de cada estado
agrup = df.groupby(['sg_uf'])['nm_meso_regiao'].agg(pd.Series.mode)
# completa os dados nulos com o termo mais frequento do estado
df.nm_meso_regiao.loc[pd.isnull(df.nm_meso_regiao)] = df.sg_uf.map(agrup)

# cria um dicionário com a micro região mais frequente de cada estado
agrup = df.groupby(['sg_uf'])['nm_micro_regiao'].agg(pd.Series.mode)
# completa os dados nulos com o termo mais frequente do micro região
df.nm_micro_regiao.loc[pd.isnull(df.nm_micro_regiao)] = df.sg_uf.map(agrup)

In [106]:
# completa os dados nulos com a mediana do valor de faturamento estimado
df.vl_faturamento_estimado_aux.fillna(df.vl_faturamento_estimado_aux.median(), inplace = True)
df.de_faixa_faturamento_estimado.fillna(df.de_faixa_faturamento_estimado.mode().values[0], inplace = True)

In [107]:
# cria um dicionário com o tipo de atividade mais frequente de cada faixa de faturamento estimado
agrup = df.groupby(['de_faixa_faturamento_estimado'])['de_nivel_atividade'].agg(pd.Series.mode)
# completa os dados nulos com o termo mais frequento do por faixa de faturamento
df.de_nivel_atividade.loc[pd.isnull(df.de_nivel_atividade)] = df.de_faixa_faturamento_estimado.map(agrup)

In [108]:
# cria um dataset com algumas colunas selecionadas para serem testadas o processo de clusterização
df_cluster = df.filter(items = ['id','de_natureza_juridica','sg_uf','de_ramo','setor','idade_empresa_anos','nm_segmento','de_nivel_atividade','nm_meso_regiao','nm_micro_regiao','qt_socios','vl_faturamento_estimado_aux'])

In [109]:
df_cluster.head(10)

,id,de_natureza_juridica,sg_uf,de_ramo,setor,idade_empresa_anos,nm_segmento,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,qt_socios,vl_faturamento_estimado_aux
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,SOCIEDADE EMPRESARIA LIMITADA,RN,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,CONSTRUCAO,ALTA,LESTE POTIGUAR,NATAL,2.0,3132172.8
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,EMPRESARIO INDIVIDUAL,PI,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,ALOJAMENTO E ALIMENTACAO,BAIXA,CENTRO NORTE PIAUIENSE,TERESINA,1.0,210000.0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,EMPRESARIO INDIVIDUAL,AM,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.093151,TRANSPORTE ARMAZENAGEM E CORREIO,MEDIA,CENTRO AMAZONENSE,MANAUS,1.0,50000.0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,EMPRESARIO INDIVIDUAL,AM,SERVICOS DIVERSOS,SERVIÇO,6.512329,OUTRAS ATIVIDADES DE SERVICOS,MEDIA,CENTRO AMAZONENSE,MANAUS,1.0,210000.0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,EMPRESARIO INDIVIDUAL,RN,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVIÇO,3.200000,ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,ALTA,LESTE POTIGUAR,NATAL,1.0,210000.0
5,529a1f5fc6f88071db7d38492a0ce7c985c77e9b1c3bb6...,SOCIEDADE EMPRESARIA LIMITADA,RN,QUIMICA-PETROQUIMICA,INDUSTRIA,38.079452,INDUSTRIAS DE TRANSFORMACAO,ALTA,OESTE POTIGUAR,PAU DOS FERROS,0.0,210000.0
6,8bf37c178778e537b108e58bf20552abe10c3a7fae129b...,EMPRESARIO INDIVIDUAL,MA,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.276712,TRANSPORTE ARMAZENAGEM E CORREIO,MEDIA,NORTE MARANHENSE,AGLOMERACAO URBANA DE SAO LUIS,1.0,50000.0
7,0114fd3fbdc347c561843cde6aece12e8e53cb5d32c77d...,EMPRESARIO INDIVIDUAL,RN,SERVICOS ADMINISTRATIVOS,SERVIÇO,0.052055,ATIVIDADES ADMINISTRATIVAS E SERVICOS COMPLEME...,ALTA,LESTE POTIGUAR,NATAL,1.0,210000.0
8,6c45aabb849a760a3a7107c7a61262642f683feb8912a4...,COOPERATIVA,AM,BENS DE CONSUMO,INDUSTRIA,12.263014,INDUSTRIAS DE TRANSFORMACAO,ALTA,CENTRO AMAZONENSE,PARINTINS,2.0,210000.0
9,2fcd678d71b04ccb77644fff57706bdc7fcfbbec6b1926...,EMPRESARIO INDIVIDUAL,RN,COMERCIO VAREJISTA,COMERCIO,17.942466,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,MEDIA,OESTE POTIGUAR,MOSSORO,1.0,210000.0


In [110]:
labelencoder =  LabelEncoder()
scaler = StandardScaler()

In [117]:
# df_cluster.de_natureza_juridica = labelencoder.fit_transform(df_cluster.de_natureza_juridica)
# df_cluster.sg_uf = labelencoder.fit_transform(df_cluster.sg_uf)
# df_cluster.de_ramo= labelencoder.fit_transform(df_cluster.de_ramo)
# df_cluster.setor = labelencoder.fit_transform(df_cluster.setor)
# df_cluster.nm_segmento = labelencoder.fit_transform(df_cluster.nm_segmento)
# df_cluster.de_nivel_atividade = labelencoder.fit_transform(df_cluster.de_nivel_atividade)
# df_cluster.nm_meso_regiao = labelencoder.fit_transform(df_cluster.nm_meso_regiao)
# df_cluster.nm_micro_regiao = labelencoder.fit_transform(df_cluster.nm_micro_regiao)


In [127]:
df_cluster.head()

,id,de_natureza_juridica,sg_uf,de_ramo,setor,idade_empresa_anos,nm_segmento,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,qt_socios,vl_faturamento_estimado_aux
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,60,4,11,CONSTRUÇÃO CIVIL,14.457534,CONSTRUCAO,0,LESTE POTIGUAR,NATAL,2.0,3132172.8
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,17,3,21,SERVIÇO,1.463014,ALOJAMENTO E ALIMENTACAO,1,CENTRO NORTE PIAUIENSE,TERESINA,1.0,210000.0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,17,1,32,SERVIÇO,7.093151,TRANSPORTE ARMAZENAGEM E CORREIO,2,CENTRO AMAZONENSE,MANAUS,1.0,50000.0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,17,1,25,SERVIÇO,6.512329,OUTRAS ATIVIDADES DE SERVICOS,2,CENTRO AMAZONENSE,MANAUS,1.0,210000.0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,17,4,27,SERVIÇO,3.200000,ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,0,LESTE POTIGUAR,NATAL,1.0,210000.0


In [136]:
list(df_cluster.setor.values)

['CONSTRUÇÃO CIVIL',
 'SERVIÇO',
 'SERVIÇO',
 'SERVIÇO',
 'SERVIÇO',
 'INDUSTRIA',
 'SERVIÇO',
 'SERVIÇO',
 'INDUSTRIA',
 'COMERCIO',
 'COMERCIO',
 'COMERCIO',
 'SERVIÇO',
 'SERVIÇO',
 'CONSTRUÇÃO CIVIL',
 'SERVIÇO',
 'SERVIÇO',
 'CONSTRUÇÃO CIVIL',
 'SERVIÇO',
 'COMERCIO',
 'SERVIÇO',
 'COMERCIO',
 'COMERCIO',
 'COMERCIO',
 'SERVIÇO',
 'SERVIÇO',
 'COMERCIO',
 'COMERCIO',
 'SERVIÇO',
 'SERVIÇO',
 'INDUSTRIA',
 'COMERCIO',
 'SERVIÇO',
 'COMERCIO',
 'COMERCIO',
 'SERVIÇO',
 'SERVIÇO',
 'COMERCIO',
 'SERVIÇO',
 'COMERCIO',
 'SERVIÇO',
 'COMERCIO',
 'SERVIÇO',
 'COMERCIO',
 'COMERCIO',
 'COMERCIO',
 'SERVIÇO',
 'SERVIÇO',
 'SERVIÇO',
 'COMERCIO',
 'SERVIÇO',
 'SERVIÇO',
 'SERVIÇO',
 'COMERCIO',
 'COMERCIO',
 'COMERCIO',
 'COMERCIO',
 'COMERCIO',
 'COMERCIO',
 'COMERCIO',
 'COMERCIO',
 'SERVIÇO',
 'SERVIÇO',
 'COMERCIO',
 'SERVIÇO',
 'CONSTRUÇÃO CIVIL',
 'SERVIÇO',
 'SERVIÇO',
 'SERVIÇO',
 'INDUSTRIA',
 'COMERCIO',
 'COMERCIO',
 'INDUSTRIA',
 'SERVIÇO',
 'COMERCIO',
 'SERVIÇO',
 'COMERCIO'

# Modelo 